In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_49.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_67_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_90.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_77.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_20_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_54.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_106_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_42.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_81.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_107.wav
/kaggle/input/shl-in

In [1]:
Train_audio_dir = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train"
Test_audio_dir  = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test"

Train_csv= "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/train.csv"
Test_csv  = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/test.csv"


In [11]:
import os

print(len(os.listdir(Train_audio_dir)))
print(os.listdir(Train_audio_dir)[:5])


409
['audio_49.wav', 'audio_77_2.wav', 'audio_90.wav', 'audio_77.wav', 'audio_66.wav']


In [12]:
import pandas as pd

train_df = pd.read_csv(Train_csv)
test_df  = pd.read_csv(Test_csv)

train_df.head()



,filename,label
0,audio_173,3.0
1,audio_138,3.0
2,audio_127,2.0
3,audio_95,2.0
4,audio_73,3.5


In [4]:
import os
import numpy as np
import pandas as pd
import librosa

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [14]:
audio_path = os.path.join(Train_audio_dir, row['filename'])


In [20]:
import os

print(os.listdir(Train_audio_dir)[:10])


['audio_49.wav', 'audio_77_2.wav', 'audio_90.wav', 'audio_77.wav', 'audio_66.wav', 'audio_54.wav', 'audio_237.wav', 'audio_42.wav', 'audio_257.wav', 'audio_256_2.wav']


In [25]:
# Remove duplicate .wav if present
train_df['filename'] = train_df['filename'].str.replace(r'(\.wav)+$', '.wav', regex=True)

print(train_df.head())



        filename  label
0  audio_173.wav    3.0
1  audio_138.wav    3.0
2  audio_127.wav    2.0
3   audio_95.wav    2.0
4   audio_73.wav    3.5


In [26]:
import os

sample_file = train_df.iloc[0]['filename']
sample_path = os.path.join(Train_audio_dir, sample_file)

print(sample_path)
print(os.path.exists(sample_path))



/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train/audio_173.wav
True


In [17]:
def extract_features(audio_path):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        features = np.hstack([
            mfcc.mean(axis=1),
            mfcc.std(axis=1)
        ])
        return features
    except Exception as e:
        return None


In [32]:
import warnings
warnings.filterwarnings("ignore")


In [33]:
X, y = [], []

for _, row in train_df.iterrows():
    audio_path = os.path.join(Train_audio_dir, row['filename'])
    features = extract_features(audio_path)

    if features is not None:
        X.append(features)
        y.append(row['label'])

import numpy as np
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)



(409, 26) (409,)


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", Ridge(alpha=1.0))
])

model.fit(X_train, y_train)

print("Validation R2:", model.score(X_val, y_val))


Validation R2: 0.00981046391739826


In [36]:
test_df = pd.read_csv(Test_csv)

test_df['filename'] = test_df['filename'].astype(str) + '.wav'

X_test = []

for _, row in test_df.iterrows():
    audio_path = os.path.join(Test_audio_dir, row['filename'])
    features = extract_features(audio_path)
    if features is not None:
        X_test.append(features)
    else:
        X_test.append(np.zeros(X.shape[1]))

X_test = np.array(X_test)

predictions = model.predict(X_test)


In [37]:
print(predictions[:5])
print(len(predictions))


[2.852219  2.175922  2.795339  3.3692856 1.8883513]
197


In [38]:
submission = pd.DataFrame({
    "filename": test_df['filename'].str.replace('.wav', '', regex=False),
    "label": predictions
})

submission.to_csv("submission.csv", index=False)


In [39]:
import os
os.listdir()



['.virtual_documents', 'submission.csv']

In [40]:
import os
os.listdir("/kaggle/working")


['.virtual_documents', 'submission.csv']